# Download data and unzip

In [1]:
!rm -rf *
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip -q nature_12K.zip
!rm nature_12K.zip 
!find . -name '.DS_Store' -type f -delete
!pip install rich wandb

# Imports

In [2]:
import yaml
import os
import cv2
import time
import glob
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
from rich import print
from pprint import pprint
from cv2 import imread, cvtColor

from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, Nadam
from tensorflow.keras.layers import  Dense, Input, InputLayer, Flatten, Conv2D, BatchNormalization, MaxPooling2D, Activation , GlobalAveragePooling2D

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2 as IRV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception


from matplotlib import pyplot as plt
%matplotlib inline

import wandb
from wandb.keras import WandbCallback

# Constants

In [3]:
ROOT_DIR = "./inaturalist_12K/"
TRAIN_DIR = "./inaturalist_12K/train/"
TEST_DIR = "./inaturalist_12K/val/"
os.environ["WANDB_API_KEY"] = "f058e3418fb166c3bee2d5131ef0bcc4b642793f"
IMAGE_SIZE = (224,224)

class_labels = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [4]:
image_shape = (224, 224)
num_classes = 10

config = {
    "epochs": 5,
    "batch_size": 64,
    "data_augmented": False,
    "model_name": 'InceptionResNetV2',

    "optimizer": 'adam',
    "learning_rate": 1e-3,
    "loss_function": 'categorical_crossentropy',
    
    "no_neurons_dense": 512,
}


# Data Generator

In [5]:
def get_data_generator(train_dir, test_dir, batch_size=64, image_shape=(224,224), data_augmentation=False):

    if data_augmentation == True:
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=0.1,
            zoom_range=0.2,
            rotation_range=10,
            horizontal_flip=True,
        )

    else:
        train_datagen = ImageDataGenerator(
            rescale=1./255, validation_split=0.1)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        subset='training',
        target_size=image_shape,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=42
    )

    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        subset='validation',
        target_size=image_shape,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=42
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=image_shape,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=42
    )

    return train_generator, validation_generator, test_generator


# Model

In [23]:
pretrained_models = {
    'Xception':          lambda: tf.keras.applications.xception.Xception(weights='imagenet', include_top=False),
    'ResNet50':          lambda: tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False),
    'InceptionV3':       lambda: tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False),
    'InceptionResNetV2': lambda: tf.keras.applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False)
}

freez_till_map = {
    'Xception':          116,
    'ResNet50':          143,
    'InceptionV3':       249,
    'InceptionResNetV2': 617
}


# Train

In [24]:
def get_model(model_name='InceptionResNetV2', no_neurons_dense=1024, freez_till=617, no_classes=10):

    pretrained_model = pretrained_models[model_name]()

    predictions = Dense(no_classes, activation='softmax')(
        Dense(no_neurons_dense, activation='relu')(
            GlobalAveragePooling2D()(
                pretrained_model.output
            )
        )
    )
    
    model = Model(inputs=pretrained_model.input, outputs=predictions)

    for layer in pretrained_model.layers:
        layer.trainable = False

    for i in range(len(model.layers)):
        if i < freez_till:
            model.layers[i].trainable = False
        else:
            model.layers[i].trainable = True

    return model

In [25]:
def train_model(model, train_data, loss_function, optimizer='adam', learning_rate=1e-3, epochs=10, val_data=None):
    if optimizer == 'adam':
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss=loss_function, metrics=['accuracy'])
    elif optimizer == 'nesterov':
        model.compile(optimizer=SGD(learning_rate=learning_rate, momentum=0.9,
                      nesterov=True), loss=loss_function, metrics=['accuracy'])
    elif optimizer == 'nadam':
        model.compile(optimizer=Nadam(learning_rate=learning_rate),
                      loss=loss_function, metrics=['accuracy'])

    print("Training Started")
    model.fit(train_data,
              epochs=epochs,
              validation_data=val_data,
              verbose=2,
              callbacks=[WandbCallback()])
    return model


In [26]:
def run(image_shape=image_shape, train_data_path=TRAIN_DIR, test_data_path=TEST_DIR, config=config):
    tf.keras.backend.clear_session()
    
    # Data generators
    train_data, val_data, test_data = get_data_generator(train_data_path, test_data_path, config['batch_size'], image_shape, config['data_augmented'])
    
    # Model
    with tf.device('/device:GPU:0'):
        model = get_model(config["model_name"], config["no_neurons_dense"], freez_till_map[config["model_name"]], num_classes)
        model = train_model(model, train_data, config['loss_function'], config['optimizer'], config['learning_rate'], config['epochs'], val_data)
    # Evaluate
    if test_data is not None:
        test_loss, test_accuracy = model.evaluate(test_data, verbose=4)
        wandb.log({
                'test_accuracy': test_accuracy, 
                'test_loss': test_loss
            })


In [ ]:
wandb.init()
run()
wandb.finish()

In [14]:
def run_sweep(data_path=ROOT_DIR):
    config = {
        "epochs": 5,
        "batch_size": 64,
        "data_augmented": False,
        "model_name": 'InceptionResNetV2',

        "optimizer": 'adam',
        "learning_rate": 1e-3,
        "loss_function": 'categorical_crossentropy',

        "no_neurons_dense": 512,
    }

    wandb.init(config=config)
    config = wandb.config

    wandb.run.name = f'TL_NM_{config.model_name}_DN_{config.no_neurons_dense}_OP_{config.optimizer}'
    wandb.run.name += '_AG' if config.data_augmented else ''

    run()
    
    wandb.finish()


In [16]:
sweep_config = {
    'name': 'CNN_pretrained',
    'method': 'grid',
#     'metric': {
#         'name': 'val_accuracy',
#         'goal': 'maximize'
#     },
    'parameters': {
        'model_name': {
            'values': ['InceptionResNetV2', 'InceptionV3', 'ResNet50', 'Xception']
        },
        'data_augmented': {
            'values': [True, False]
        },
        'no_neurons_dense': {
            'values': [512, 1024]
        },
        'optimizer': {
#             'values': ['adam', 'nesterov', 'nadam']
            'values': ['adam']
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="CS6910_Assignment2")
wandb.agent(sweep_id, run_sweep, count=1)